In [ ]:
!pip install tensorflow opencv-python matplotlib


TESTING

In [ ]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import cv2
import imghdr

In [ ]:
image_demo = cv2.imread(os.path.join('data','happy','1-2.jpg'))


In [ ]:
image_demo.shape  #pixels height, pixels width, 3 channels means its coloured

In [ ]:
plt.imshow(cv2.cvtColor(image_demo,cv2.COLOR_BGR2RGB))
plt.show()

REMOVE DODGY IMAGES

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir = 'data'

In [ ]:
image_exts = ['jpg', 'jpeg', 'png','bmp']

In [ ]:
for image_class in os.listdir(data_dir):
    image_class_path = os.path.join(data_dir, image_class)
    if os.path.isdir(image_class_path):
        for image in os.listdir(os.path.join(image_class_path)):
            image_path = os.path.join(image_class_path, image)
            try:
                img = cv2.imread(image_path) # reads the image in opencv
                tip = imghdr.what(image_path) # checks the extensions
                if tip not in image_exts:
                    print("Issue with image extension".format(image_path))
                    os.remove(image_path)
            except Exception as e:
                print('Issue with image {}'.format(image_path))
                # os.remove(image_path)

LOAD THE DATA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
batch[0]


In [ ]:
batch[1]

In [ ]:
fig, ax = plt.subplots(ncols = 4, figsize = (20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])


SCALE DATA

In [ ]:
data = data.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))

In [ ]:
scaled_iterator = data.as_numpy_iterator()
batch = scaled_iterator.next()

In [ ]:
batch[0].max()


In [ ]:
fig, ax = plt.subplots(ncols = 4, figsize = (20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

SPLIT THE DATA

In [ ]:
len(data) ##number of batches

In [ ]:
train_size = int(len(data)*.7) ##used to train model
val_size = int(len(data)*.2) ##used to evalute model while training to fine tune the model
test_size = int(len(data)*.1)

In [ ]:
train_size + val_size + test_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

BUILD A NETWORK

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation = 'relu', input_shape = (256, 256, 3))) # inputshape is only in the first layer and '1' is the stride
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D()) 

model.add(Conv2D(16, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D()) 

model.add(Flatten())

model.add(Dense(256, activation ='relu'))
model.add(Dense(1, activation ='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

TRAIN DATA

In [ ]:
logdir = 'logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = logdir) ##used to log data for later verification

In [ ]:
hist = model.fit(train, epochs=20, validation_data = val, callbacks = [tensorboard_callback])

PLOT

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color = 'teal', label = 'Loss')
plt.plot(hist.history['val_loss'], color = 'orange', label = 'Validation Loss')
plt.title('LOSS')
plt.legend(loc = "upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color = 'teal', label = 'Accuracy')
plt.plot(hist.history['val_accuracy'], color = 'orange', label = 'Validation Accuracy')
plt.title('ACCURACY')
plt.legend(loc = "upper left")
plt.show()

EVALUATE MODEL

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pe = Precision()
re = Recall()
ba = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    pe.update_state(y, yhat)
    re.update_state(y, yhat)
    ba.update_state(y, yhat)

In [ ]:
pe_result = pe.result().numpy()
re_result = re.result().numpy()
ba_result = ba.result().numpy()

In [ ]:
print(pe_result)
print(re_result)
print(ba_result)

TEST

In [ ]:
import cv2

In [ ]:
img = cv2.imread('happy.jpg')
plt.imshow(img)
plt.show()

In [ ]:
resize_img = tf.image.resize(img, (256, 256))
plt.imshow(resize_img.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize_img/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5: 
    print(f'Predicted class is Sad')
else:
    print(f'Predicted class is Happy')

SAVE THE MODEL

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','imageclassifier2.h5'))

In [ ]:
new_model = load_model('imageclassifier2.h5')

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))